<a href="https://colab.research.google.com/github/omar-oahmad/rag/blob/main/dsiyp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install chromadb==0.4.18 langchain==0.0.349 sentence_transformers tiktoken==0.5.2

import chromadb
from chromadb.utils import embedding_functions
from langchain.embeddings import HuggingFaceEmbeddings
import os

os.environ['HUGGINGFACEHUB_API_TOKEN'] = ''

chroma_client = chromadb.PersistentClient(path='Context')

#create embedding func
embedding = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="sentence-transformers/all-MiniLM-L6-v2")
# embedding = HuggingFaceEmbeddings()

#create collection
collection = chroma_client.get_or_create_collection(name='Context', embedding_function=embedding)

#load text file & segment
with open('../sherlock.txt', 'r') as file:
    data = file.read().replace('\n', '.').split('.')

#add segments to collection
collection.add(
    documents = data,
    embeddings = [embedding.embed_query(x) for x in data],
    ids = ['id'+str(x) for x in range (len(data))]
)


FileNotFoundError: [Errno 2] No such file or directory: '../sherlock.txt'